# SQL Notebook

### **Return the number of distinct customers handled by each employee**

In [ ]:
select 
    count (distinct c.customer_id) as distinct_customers, 
    s.staff_id, 
    CONCAT (s.first_name, SPACE(1), s.last_name) as full_name  
from 
    sales.customers as c
    inner join sales.orders as o on c.customer_id = o.customer_id
    inner join sales.staffs as s on o.staff_id = s.staff_id
group by 
    s.staff_id, 
    CONCAT (s.first_name, SPACE(1), s.last_name)

### **Return the customers from NY that also placed orders**

In [ ]:
select
    *
from 
    sales.customers as c
where 
    c.[state] = 'NY'
        AND exists (select*
                    from sales.orders as o
                    where o.customer_id = c.customer_id)

### **Find all employees with no orders**

In [ ]:
select 
	CONCAT(s.first_name, ' ' , s.last_name) as Full_name, 
	o.order_id
from sales.staffs as s
	left outer join sales.orders as o on s.staff_id = o.staff_id -- to return all records from the left side 	
where 
	o.order_id is null -- to return only the employees with no orders since outer rows are represented by null marks
group by  
	CONCAT(s.first_name, ' ' , s.last_name),
	o.order_id

### **For every employee, return the number of days passed between their first and last order**

In [ ]:
SELECT 
    CONCAT(s.first_name, space(1), s.last_name) as full_name,
    s.staff_id, 
    MIN(o.order_date) as first_order, 
    MAX(o.order_date) as last_order, 
    DATEDIFF(DAY,MIN(o.order_date),MAX(o.order_date)) as day_difference
-- DATEDIFF can return nulls in case the employees have no orders. These nulls are returned as a consequence of the outer join. 
-- We can wrap the DATEDIFF function in COALESCE/ISNULL to replace null values.
from 
    sales.staffs as s
    left outer join sales.orders as o on s.staff_id = o.staff_id
group by 
    s.staff_id,
    CONCAT(s.first_name, space(1), s.last_name)
order by 
    day_difference desc

### **Compute the running total for each customer**

In [1]:
--> Running totals partitioned by customer
select 
	o.customer_id,
    o.order_date,
	datename(mm,o.order_date) as month_name,
	SUM(oi.quantity*list_price) over (partition by o.customer_id order by o.order_date desc
									  ROWS BETWEEN UNBOUNDED PRECEDING -- frame
									  AND CURRENT ROW) as Running_totals    
from 
	sales.order_items as oi
	inner join sales.orders as o on oi.order_id = o.order_id

(4722 rows affected)

Total execution time: 00:00:00.266

customer_id,order_date,month_name,Running_totals
1,2018-11-18,November,4999.99
1,2018-11-18,November,5639.97
1,2018-04-18,April,6739.95
1,2018-04-18,April,13239.94
1,2018-04-18,April,13709.93
1,2018-04-18,April,17507.93
1,2018-04-18,April,20707.92
1,2016-12-09,December,26707.90
1,2016-12-09,December,27247.88
1,2016-12-09,December,27547.87


### **Return the difference between the current and the previous customer's order, first value and the last value**

In [ ]:
with orders as
(
	select 
		CONCAT(c.first_name, ' ' , c.last_name) as full_name, 
	    oi.quantity * oi.list_price * (1-oi.discount) as order_price, -- price of a single order
	    c.customer_id as custid, 
	    o.order_id as ordid	   
	from 
		sales.customers as c
		inner join sales.orders as o on c.customer_id = o.customer_id
		inner join sales.order_items as oi on o.order_id = oi.order_id
	group by 
		CONCAT(c.first_name, ' ' , c.last_name),
	    oi.quantity*oi.list_price*(1-oi.discount),
	    c.customer_id,
	    o.order_id 	
), -- this query computes the order price for each customer and order

Previous as
(
	select *,	
	(order_price - LAG(order_price) over (partition by custid order by ordid)) as difference,
	-- difference between the current customer's order and the previous one
			
	FIRST_VALUE(order_price) over (PARTITION by custid order by ordid, order_price 
	ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as first_val,
	-- first value

	LAST_VALUE (order_price) over (PARTITION by custid order by ordid, order_price
							 	   ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) 
								   AS last_val
	-- last value
from orders 
)

select full_name, 
	   order_price,
	   difference,
	   first_val,
	   last_val
from Previous


### **Return the customer who bought the most from the Trek brand**

In [ ]:
with trek_orders as 
(
select count(o.order_id) as number_of_orders,
	   concat(c.first_name, ' ' , c.last_name) as full_name, 
	   b.brand_name as brand, 
	   c.customer_id, 
	   b.brand_id, 
	   o.order_id 
from sales.order_items as oi
	inner join production.products as p on oi.product_id = p.product_id
	inner join production.brands as b on p.brand_id = b.brand_id
	inner join sales.orders as o on oi.order_id = o.order_id
	inner join sales.customers as c on o.customer_id = c.customer_id
where b.brand_name = 'Trek'
group by c.customer_id, 
		 o.order_id, 
		 concat(c.first_name, ' ' , c.last_name), 
		 b.brand_name, 
		 b.brand_id
),

brand_rank as 
(
select*, RANK () over (order by number_of_orders desc) as rang 
from trek_orders
)

select rang, 
	   full_name, 
	   number_of_orders, 
	   brand
from brand_rank as br
where rang = 1

### **Return the top 3 most sold products for the most sold category in year 2016**

In [ ]:
with category_totals as 
(
select c.category_name as category, 
	   c.category_id as category_id,
	   sum(oi.quantity) as total_category_quantity, -- total quantity for category
	   rank() over (order by sum(oi.quantity) desc) as category_rank -- category rank
from sales.order_items as oi
	inner join production.products as p on oi.product_id = p.product_id
	inner join production.categories as c on p.category_id = c.category_id
group by c.category_name, 
	     c.category_id
), 

product as 
(
	select p.product_id, 
		p.product_name,
		category, 
		sum(oi.quantity) as product_quantity, -- total quantity for product
		rank() over (order by sum(oi.quantity) desc) as product_rank -- product rank
	from sales.order_items as oi
		inner join production.products as p on oi.product_id = p.product_id
		inner join category_totals as ct on p.category_id = ct.category_id
	where category_rank = 1 -- filters only the products for which the category rank is 1
	group by p.product_id, 
		p.product_name,
		category
)

select*
from product
where product_rank <= 3

### **Find the top 5 customers with the biggest orders in year 2017., given that the order status is 4 (delivered)**

In [12]:
with totals as 
(
select concat(c.first_name, ' ' , c.last_name) as full_name, 
	   o.order_id, 
	   c.customer_id, 
	   sum(oi.quantity*oi.list_price*(1-oi.discount)) as order_price -- price of a single order when discount is taken into account (net price)
from sales.order_items as oi
	inner join sales.orders as o on oi.order_id = o.order_id
	inner join sales.customers as c on o.customer_id = c.customer_id
where o.order_date between '2017-01-01' AND '2017-12-31' AND o.order_status = 4
group by concat(c.first_name, ' ' , c.last_name),
		  o.order_id, 
		  c.customer_id
), 
rang as 
(
select*, ROW_NUMBER() over (order by order_price desc) as ranking
from totals
)

select full_name, ranking, order_price
from rang
where ranking <= 5

(5 rows affected)

Total execution time: 00:00:00.084

full_name,ranking,order_price
Melanie Hayes,1,27050.7182
Abram Copeland,2,24607.0261
Cindi Larson,3,20177.7457
Adena Blake,4,19329.9492
Penny Acevedo,5,18670.9288


**Between the customers who made at least 3 orders ruturn the ones whose order processing (average) took the most time**

In [ ]:
with Num_orders as 
	(
	select CONCAT(c.first_name, ' ' , c.last_name) as full_name,
		   count(o.order_id) as order_count,
		   avg(DATEDIFF(day,o.order_date, o.shipped_date)) as processing_time -- average difference in days between order_date and shipped_date
	  
	from sales.orders as o
		   inner join sales.customers as c on o.customer_id = c.customer_id
	group by  CONCAT(c.first_name, ' ' , c.last_name)		  
	having count(o.order_id) >= 3 -- filtering groups with more than 3 orders
	),
		  
Rang as
	(
	select*, dense_rank() over (order by processing_time desc, order_count desc) as time_rank -- dense rank in case of ties
	from Num_orders 
	)
select full_name, 
	   order_count, 
	   processing_time,
	   time_rank
from   Rang
where  time_rank = 1

### <span style="font-size: 16.38px;"><b>For every store return the current stock level for the most sold product</b></span>

In [ ]:
with totals as -- this query returns the total quantity sold for each product and its store
(
	select 
		st.store_id, 
		p.product_id, 
		st.store_name as store, 
		p.product_name as product,
		SUM(oi.quantity) as total_sold
			
	from sales.order_items as oi
		inner join sales.orders as o on oi.order_id = o.order_id
		inner join sales.stores as st on o.store_id = st.store_id 
		inner join production.products as p on oi.product_id = p.product_id

	group by 
		st.store_id,
		p.product_id, 
		st.store_name, 
		p.product_name
), 
rang as 
(
	select 
		*, 
		ROW_NUMBER() over (partition by store order by total_sold desc) as rang -- ranking by the total quantity sold
	from
		totals
)

select 
	store, 
	product, 
	rang, 
	total_sold, 
	stc.quantity as stock
from
	rang as r
	inner join production.stocks as stc on r.store_id = stc.store_id and r.product_id = stc.product_id
where 
	rang = 1 

### **Return the grand total for all orders and the customer total**

In [ ]:
select o.customer_id,
	   o.order_id,
	   oi.quantity * oi.list_price * (1-oi.discount) as order_price,
	   SUM ((oi.quantity * list_price * (1-oi.discount))) over () as grand_total, -- grand total of all orders
       -- the total value calculated for all rows
	   SUM ((oi.quantity * list_price * (1-oi.discount))) over (partition by o.customer_id) as customer_total -- customer total (the sum of all orders for a particular customer)
       -- total value for all rows that have the same custid value as in the current row
from sales.order_items as oi
	inner join sales.orders as o on oi.order_id = o.order_id

### **Return the percentage of the current value in the grand total / customer total**

In [ ]:
with a as
(
select oi.order_id as orderID, 
	   c.customer_id as custID, 
	   (oi.quantity * oi.list_price * (1-oi.discount)) as order_price,
	   100 * (oi.quantity * oi.list_price * (1-oi.discount)) / sum(oi.quantity * oi.list_price * (1-oi.discount)) over () as pct_all, 
	   -- % of the current value in the grand total
	   100 * (oi.quantity * oi.list_price * (1-oi.discount)) / sum(oi.quantity * oi.list_price * (1-oi.discount)) over (partition by c.customer_id) as pct_cust
	   -- % of the current value in the customer total 
from sales.order_items as oi
	inner join sales.orders as o on o.order_id = oi.order_id
	inner join sales.customers as c on o.customer_id = c.customer_id
)

select orderID,
		custID, 
		order_price,
		CAST(pct_all as numeric (5,2)) as pct_grand_total,
		CAST(pct_cust as numeric (5,2)) as pct_cust_total
from a

### **Return information about all employees and find the superior and its subordinates (superior is the one without manager, i.e. where manger\_id is NULL)**

Recursive CTE aimed at querying hierarchical data

In [3]:

with no_manager as 

(
select s.staff_id, -- anchor member returns the superior staff member who is a top manager
	   s.manager_id,
	   s.first_name, 
	   s.last_name
from sales.staffs as s
	   where s.manager_id is null  -- manager_id being NULL represents a staff member who has no manager (superior)

union all 

select s.staff_id, -- recursive member returns subordinates of the top manager 
	   s.manager_id,
	   s.first_name, 
	   s.last_name
from no_manager as nm
	inner join sales.staffs as s on s.manager_id = nm.staff_id 
)

select staff_id, 
	   manager_id,
	   first_name, 
	   last_name

from no_manager
order by staff_id

(10 rows affected)

Total execution time: 00:00:01.330

staff_id,manager_id,first_name,last_name
1,NULL,Fabiola,Jackson
2,1,Mireya,Copeland
3,2,Genna,Serrano
4,2,Virgie,Wiggins
5,1,Jannette,David
6,5,Marcelene,Boyer
7,5,Venita,Daniel
8,1,Kali,Vargas
9,7,Layla,Terrell
10,7,Bernardine,Houston


### **Return the days of the week**

In [ ]:
WITH cte_numbers 
AS 
(
    SELECT 
        0 as n, 
        DATENAME (DW, 0) as weekday -- this query returns Monday
    UNION ALL
    SELECT    
        n + 1, 
        DATENAME (DW, n + 1) -- this query adds one day until n reaches 6 (Sunday)
    FROM    
        cte_numbers
    WHERE n < 6
)
SELECT 
    weekday
FROM 
    cte_numbers;

### **For every store return the current stock levels for the most sold product in that store**

In [ ]:
with stock as
	(
	select st.store_id, 
		    p.product_id, 
		    st.store_name as store, 
		    p.product_name as product,
		    SUM(oi.quantity) as total_quantity
from sales.order_items as oi
		inner join sales.orders as o on oi.order_id = o.order_id
		inner join sales.stores as st on o.store_id = st.store_id 
		inner join production.products as p on oi.product_id = p.product_id
group by st.store_id,
		p.product_id, 
		st.store_name, 
		p.product_name
), 
Rang as 
(
select *, 
    ROW_NUMBER() over (PARTITION by store order by total_quantity desc) as rank
from stock
)

select store, product, rank, total_quantity
from Rang as r
    --inner join production.stocks as stc on r.store_id = stc.store_id and r.product_id = stc.product_id
where rank = 1

### **For the customers who made at least 3 orders, return the top 5 with the biggest average orders**

In [ ]:
with Orders as 
(
    select  
        count(o.order_id) as Order_number,
		CONCAT(c.first_name, ' ', c.last_name) as Full_name,
		avg(oi.list_price * oi.quantity * (1-oi.discount)) as avg_order, -- average order 
		o.customer_id as custid,
		o.order_id as orderid
    from 
        sales.orders as o
		inner join sales.customers as c on o.customer_id=c.customer_id
		inner join sales.order_items as oi on o.order_id=oi.order_id
    group by 
        CONCAT(c.first_name, ' ', c.last_name),
		o.customer_id,
		o.order_id
    having count(o.order_id) >= 3 -- returning groups with at least 3 orders
), 

Customers as 
(
    select 
        *,
		Row_number() over(order by avg_order desc) as rang
    from Orders
)

select	avg_order, 
		Full_name,
		order_number,
		rang
from Customers 
where rang <= 5
order by avg_order desc

### **For the most sold brand return the top 3 most sold products**

In [1]:
with brand_totals as -- brand level
(
	select  
		pb.brand_id, 
		pb.brand_name as brand, 
		sum(oi.quantity) as brand_quantity, 
		rank() over (order by sum(oi.quantity) desc) as brand_rank
	from sales.order_items as oi
		inner join production.products as p on oi.product_id = p.product_id
		inner join production.brands as pb on p.brand_id = pb.brand_id
	group by 
		pb.brand_id, 
		pb.brand_name
), 

product_totals as -- product level
(
	select 
		brand, 
		p.product_id, 
		p.product_name, 
		sum (oi.quantity) as product_quantity, 
		rank() over (order by sum (oi.quantity) desc) as product_rank
	from 
		sales.order_items as oi
		inner join production.products as p on oi.product_id = p.product_id
		inner join brand_totals as bt on p.brand_id = bt.brand_id
	where
		brand_rank = 1 -- returning the most sold brand
	group by 
		brand, 
		p.product_id, 
		p.product_name
)

select
	*
from product_totals 
where 
	product_rank <= 3 -- returning top 3 products from the most sold brand

(3 rows affected)

Total execution time: 00:00:00.157

brand,product_id,product_name,product_quantity,product_rank
Electra,13,Electra Cruiser 1 (24-Inch) - 2016,157,1
Electra,16,Electra Townie Original 7D EQ - 2016,156,2
Electra,23,Electra Girl's Hawaii 1 (20-inch) - 2015/2016,154,3
